In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn import Linear, Dropout, LayerNorm
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SAGEConv, global_add_pool, GraphConv
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import ReduceLROnPlateau

import networkx as nx
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import seaborn as sns
import random
import uproot

In [2]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
path_to_tree = "DecayTreeTuple/tuple"

def rename_columns(df):
    new_columns = {}
    for col in df.columns:
        if "_BDTTag_" in col:
            new_columns[col] = col.replace("_BDTTag_", "_SVtag_")
    df.rename(columns=new_columns, inplace=True)

# maybe don't need
def extract_float(awk_array):
    return awk_array[0] if len(awk_array) > 0 else None

In [ ]:
# Read the ROOT file
tree = uproot.open("Dijet_bb_pt20_50_dw.root" + ":" + path_to_tree)
bb_df = tree.arrays(tree.keys(), library="pd", entry_start=0, entry_stop=100000) 

# Clean column names and remove unwanted columns
bb_df.columns = [col.rstrip('_') if col.endswith('_') else col for col in bb_df.columns]
rename_columns(bb_df)
columns_to_remove = [col for col in bb_df.columns if (col.startswith("H_10") or col.endswith("COV") or ("Decision" in col) or col.endswith("Tag") or col.endswith("NbTag") )]
bb_df.drop(columns=columns_to_remove, inplace=True)
for col in bb_df.columns:
    if "SVtag" in col:
        bb_df[col] = bb_df[col].apply(extract_float)
tree.close()

# Read the ROOT file
tree = uproot.open("Dijet_cc_pt20_50_dw.root" + ":" + path_to_tree)
cc_df = tree.arrays(tree.keys(), library="pd", entry_start=0, entry_stop=100000)

# Clean column names and remove unwanted columns
cc_df.columns = [col.rstrip('_') if col.endswith('_') else col for col in cc_df.columns]
rename_columns(cc_df)
columns_to_remove = [col for col in cc_df.columns if (col.startswith("H_10") or col.endswith("COV") or ("Decision" in col) or col.endswith("Tag") or col.endswith("NbTag") )]
cc_df.drop(columns=columns_to_remove, inplace=True)
for col in cc_df.columns:
    if "SVtag" in col:
        cc_df[col] = cc_df[col].apply(extract_float)
tree.close()

# Read the ROOT file
tree = uproot.open("Dijet_qq_pt20_50_dw.root" + ":" + path_to_tree)
qq_df = tree.arrays(tree.keys(), library="pd", entry_start=0, entry_stop=100000) 

# Clean column names and remove unwanted columns
qq_df.columns = [col.rstrip('_') if col.endswith('_') else col for col in qq_df.columns]
rename_columns(qq_df)
columns_to_remove = [col for col in qq_df.columns if (col.startswith("H_10") or col.endswith("COV") or ("Decision" in col) or col.endswith("Tag") or col.endswith("NbTag") )]
qq_df.drop(columns=columns_to_remove, inplace=True)
for col in qq_df.columns:
    if "SVtag" in col:
        qq_df[col] = qq_df[col].apply(extract_float)
tree.close()

/data/home/pesticci/.conda/envs/gnn_env/lib/python3.9/site-packages/uproot/interpretation/library.py:795: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[name] = series[name]
/data/home/pesticci/.conda/envs/gnn_env/lib/python3.9/site-packages/uproot/interpretation/library.py:795: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[name] = series[name]
/data/home/pesticci/.conda/envs/gnn_env/lib/python3.9/site-packages/uproot/interpretation/library.py:795: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [ ]:
# Read the ROOT file
tree = uproot.open("Dijet_bb_pt50_dw.root" + ":" + path_to_tree)
bb50_df = tree.arrays(tree.keys(), library="pd", entry_start=0, entry_stop=100000) 

# Clean column names and remove unwanted columns
bb50_df.columns = [col.rstrip('_') if col.endswith('_') else col for col in bb50_df.columns]
rename_columns(bb50_df)
columns_to_remove = [col for col in bb50_df.columns if (col.startswith("H_10") or col.endswith("COV") or ("Decision" in col) or col.endswith("Tag") or col.endswith("NbTag") )]
bb50_df.drop(columns=columns_to_remove, inplace=True)
for col in bb50_df.columns:
    if "SVtag" in col:
        bb50_df[col] = bb50_df[col].apply(extract_float)
tree.close()

# Read the ROOT file
tree = uproot.open("Dijet_cc_pt50_dw.root" + ":" + path_to_tree)
cc50_df = tree.arrays(tree.keys(), library="pd", entry_start=0, entry_stop=100000)

# Clean column names and remove unwanted columns
cc50_df.columns = [col.rstrip('_') if col.endswith('_') else col for col in cc50_df.columns]
rename_columns(cc50_df)
columns_to_remove = [col for col in cc50_df.columns if (col.startswith("H_10") or col.endswith("COV") or ("Decision" in col) or col.endswith("Tag") or col.endswith("NbTag") )]
cc50_df.drop(columns=columns_to_remove, inplace=True)
for col in cc50_df.columns:
    if "SVtag" in col:
        cc50_df[col] = cc50_df[col].apply(extract_float)
tree.close()

# Read the ROOT file
tree = uproot.open("Dijet_qq_pt50_dw.root" + ":" + path_to_tree)
qq50_df = tree.arrays(tree.keys(), library="pd", entry_start=0, entry_stop=100000) 

# Clean column names and remove unwanted columns
qq50_df.columns = [col.rstrip('_') if col.endswith('_') else col for col in qq50_df.columns]
rename_columns(qq50_df)
columns_to_remove = [col for col in qq50_df.columns if (col.startswith("H_10") or col.endswith("COV") or ("Decision" in col) or col.endswith("Tag") or col.endswith("NbTag") )]
qq50_df.drop(columns=columns_to_remove, inplace=True)
for col in qq50_df.columns:
    if "SVtag" in col:
        qq50_df[col] = qq50_df[col].apply(extract_float)
tree.close()

/data/home/pesticci/.conda/envs/gnn_env/lib/python3.9/site-packages/uproot/interpretation/library.py:795: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[name] = series[name]
/data/home/pesticci/.conda/envs/gnn_env/lib/python3.9/site-packages/uproot/interpretation/library.py:795: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[name] = series[name]
/data/home/pesticci/.conda/envs/gnn_env/lib/python3.9/site-packages/uproot/interpretation/library.py:795: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [ ]:
# Define DataFrames and weights in a dictionary
dataframes = {
    "bb_df": (bb_df, 1),
    "cc_df": (cc_df, 1),
    "qq_df": (qq_df, 1),
    "bb50_df": (bb50_df, 0.03073569429),
    "cc50_df": (cc50_df, 0.02880875704),
    "qq50_df": (qq50_df, 0.02991944973),
}

# Apply weights and add truth labels
for name, (df, weight) in dataframes.items():
    df['weights'] = weight
    if "bb" in name:
        df['Class'] = 0
    elif "cc" in name:
        df['Class'] = 1
    elif "qq" in name:
        df['Class'] = 2

combined_dataframes = []
for data in ["bb", "cc", "qq"]:
    full_df = pd.concat([dataframes[f"{data}_df"][0], dataframes[f"{data}50_df"][0]])
    combined_dataframes.append(full_df)

combined_df = pd.concat(combined_dataframes).loc[:, lambda df: df.columns.str.contains("Jet0|Class|weights")]

In [ ]:
combined_df = combined_df.reset_index(drop=True)
combined_df.to_csv("combined_data.csv", index=False)